# Proteomics analysis

**Goal**: get peptides whose expression levels are significantly varying within groups

### Imports and config loading

In [108]:
import os
import sys
import copy
from pathlib import Path
import yaml
from dotenv import load_dotenv
from datetime import datetime
from typing import Literal

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import probplot, shapiro, f_oneway, kruskal, gaussian_kde
from statsmodels.stats.multitest import multipletests
from sklearn.decomposition import PCA

pd.set_option('future.no_silent_downcasting', True)


class DotDict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__


def _to_dotdict(x):
    return DotDict({k: _to_dotdict(v) if isinstance(v, dict) else v for k, v in x.items()})


def _detect_project_root():
    candidates = []
    try:
        nb_path = Path(__file__).resolve()
        candidates.extend([nb_path.parent, *nb_path.parents])
    except NameError:
        pass
    cwd = Path.cwd()
    candidates.extend([cwd, *cwd.parents])

    visited = set()
    for candidate in candidates:
        if not candidate:
            continue
        resolved = candidate.expanduser().resolve()
        if resolved in visited:
            continue
        visited.add(resolved)
        if (resolved / "params").is_dir() and (resolved / "scripts").is_dir():
            return resolved

    raise RuntimeError("Could not locate project root.")


PROJECT_ROOT = _detect_project_root()
os.chdir(PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

load_dotenv(PROJECT_ROOT / ".env")


def _load_params(params_id: str):
    candidates = sorted(Path("params").glob(f"{params_id}_*.yml"))
    if not candidates:
        raise FileNotFoundError(f"No params YAML found for ID {params_id} in ./params")
    with open(candidates[0], "r", encoding="utf-8") as f:
        return yaml.safe_load(f) or {}


def _deep_merge(base: dict, extra: dict):
    for key, value in extra.items():
        if (
            key in base
            and isinstance(base[key], dict)
            and isinstance(value, dict)
        ):
            _deep_merge(base[key], value)
        else:
            base[key] = copy.deepcopy(value)
    return base


PROJECT_ID = os.getenv("PROJECT_ID", "p02")
PROJECT_NAME = os.getenv("PROJECT_NAME", "peptide_analysis")
PARAMS_ID = os.getenv("PARAMS_ID", "000")
PARAMS_NAME = os.getenv("PARAMS_NAME", "default")
FILE_PATH_INPUT = os.getenv("FILE_PATH_INPUT", "./data")
FILE_PATH_OUTPUT = os.getenv("FILE_PATH_OUTPUT", "./results")

params_prot = _load_params(PARAMS_ID)

cfg = DotDict(
    {
        "FILE_PATH_INPUT": FILE_PATH_INPUT,
        "FILE_PATH_OUTPUT": FILE_PATH_OUTPUT,
        **_to_dotdict(params_prot),
    }
)


### Creating run folder and configuring saving of the results

In [109]:
try:
    stamp = datetime.now().strftime("%Y%m%d-%H%M%S")  # avoid ':' for Windows
    OUTPUT_DIR = os.path.join(FILE_PATH_OUTPUT, f"run_{PARAMS_ID}_{stamp}")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
except Exception as e:
    print(f"Failed to create output directory: {e}")

OutputType = Literal[
    "raw",
    "prep",
    "norm",
    "stats",
    "ml",
    "viz",
    "fig",
    "output",
    "report",
]

def get_output_file_name(step: str, type: OutputType, description: str, ext: str, cfg=None, OUTPUT_DIR=OUTPUT_DIR) -> str:
    date = pd.Timestamp.now().strftime("%Y%m%d")
    return os.path.join(OUTPUT_DIR, f"{PROJECT_ID}_{PROJECT_NAME}_{PARAMS_ID}_{PARAMS_NAME}_{step}_{type}_{description}_{date}.{ext}")

### Save figure function for notebook

In [110]:
def save_figure(fig, step: str, description: str, cfg, output_dir: str):
    fig.show()
    file_name = get_output_file_name(step, "fig", description, "png")
    fig.write_image(file_name)
    fig.write_html(file_name.replace(".png", ".html"))
    print(f"Figure saved to {file_name}")

### Loading file to dataframe

In [111]:
with pd.ExcelFile(os.path.join(FILE_PATH_INPUT, cfg.data.file_name_input)) as xls:
    sheet_names = xls.sheet_names
    final_sheets = [name for name in sheet_names if name.upper().startswith("FINAL")]

    if not final_sheets:
        raise ValueError(f"No sheet starting with 'FINAL' found in {cfg.data.file_name_input}. Available sheets: {sheet_names}")
    
    df = pd.read_excel(xls, sheet_name=final_sheets[0])
    print(f"Loaded sheet '{final_sheets[0]}' from {cfg.data.file_name_input}")

# Fixing double-row header
df = df[1:]

Loaded sheet 'FINAL results in mg-l of plasma' from 250930_PD_and_RBD_cohort_report.xlsx


### Step 1: Dataframe cleaning and preview

In [112]:
# Forward fill missing values (aka fixing merged cells with protein names)
df["Protein"] = df["Protein"].ffill()
print(f"Successfully performed forward fill on missing values.")

print("Total number of peptides before filtering:", len(df))

sample_prefix_arr = ("BIOPD", "RBD", "CON")
value_cols = [c for c in df.columns if c.startswith(sample_prefix_arr)]

df[value_cols] = df[value_cols].replace("<LOQ", 0.0).astype(float)

groups = {
    "PD": [c for c in value_cols if c.startswith("BIOPD")],
    "RBD": [c for c in value_cols if c.startswith("RBD")],
    "CON": [c for c in value_cols if c.startswith("CON")]
}

print(f"Step 1: Preview of the dataframe:")
df


Successfully performed forward fill on missing values.
Total number of peptides before filtering: 62
Step 1: Preview of the dataframe:


,Protein,Peptide,BIOPD01-PD,BIOPD02-PD,BIOPD03-PD,BIOPD04-PD,BIOPD05-PD,BIOPD06-PD,BIOPD07-PD,BIOPD09-PD,...,CON70-Control,CON72-Control,CON73-Control,CON74-Control,CON75-Control,CON77-Control,CON78-Control,CON79-Control,CON80-Control,CON64-Control
1,IgA1,TPLTATLSK,0.000000,1008.173462,627.100703,1322.910475,1497.072894,633.156067,474.088467,1433.073280,...,858.978927,427.572415,712.968236,751.394792,514.873551,817.592085,1785.952282,857.973511,931.553940,537.146821
2,A1AT iso 1,AVLTIDEK,1117.945546,863.326607,768.614780,1029.697607,833.145831,856.083694,873.374614,905.692088,...,1690.129775,1111.737753,1149.619594,1316.591069,909.570230,1118.431523,1061.172706,1018.459433,740.122801,960.775234
3,A1AT iso 1,FNKPFVFLMIEQNTK,0.000000,2264.803599,1080.854909,1402.628036,2087.625472,2333.459013,2223.153954,2382.198032,...,2851.178265,3222.218775,3673.092863,1862.792925,2879.487229,3366.295919,2875.762532,2884.315038,1104.520977,2767.834097
4,A1AT iso 1+2+3,FLENEDR,1190.833778,917.718956,821.651781,1121.922693,869.033112,926.842782,918.980137,938.705144,...,1861.883496,1220.338817,1226.625538,1402.171518,986.338584,1120.723101,1096.162319,1092.841872,831.666651,1012.624829
5,A1AT iso 1+2,LSITGTYDLK,1558.657170,1214.347809,1035.620134,1430.087299,1086.829822,1254.942457,1138.985853,1227.036130,...,2370.161075,1500.912666,1588.079657,1854.175925,1277.894195,1485.990038,1457.769701,1375.507275,982.265755,1323.073487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,KNG1 iso 1+2+3,YFIDFVAR,54.343804,63.374759,58.456167,60.535882,46.291841,40.323892,45.390583,57.749842,...,101.153628,50.985120,69.977964,57.011210,67.615565,66.529214,67.427357,56.824651,55.025900,67.144268
59,KNG1 iso 1+2+3,TVGSDTFYSFK,104.852473,88.347667,92.692195,103.299297,64.262951,65.766334,74.087074,96.987379,...,147.741321,78.359057,107.254085,87.544891,90.815523,100.053713,102.554876,88.636001,91.140682,100.513461
60,LBP,ITLPDFTGDLR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.804026,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
61,LBP,LAEGFPLPLLK,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Step 2: Removing samples with more than specified threshold of zero values (<LOQ; default: 60%)

In [113]:
zero_threshold = cfg.preprocessing.zero_threshold if cfg.preprocessing.zero_threshold is not None else 0.6

zero_fractions = (df.eq(0.0) | df.isna()).sum(axis=1) / len(df)

df_filtered = df[zero_fractions <= zero_threshold]
print(f"Step 2: Filtered peptides that have more than %d%% of '<LOQ': kept %d of %d" % (zero_threshold * 100, len(df_filtered), len(df)))
df = df_filtered.copy()


Step 2: Filtered peptides that have more than 60% of '<LOQ': kept 40 of 62


### Step 3: Imputing remaining '<LOQ' values with LOQ/2 for each peptide

In [114]:
for col in value_cols:
    min_detected = df[col].min(skipna=True)
    impute_value = min_detected / 2
    df.fillna({col: impute_value}, inplace=True)

print(f"Step 3: Imputed remaining '<LOQ' values with LOQ/2 for each peptide.")

# save intermediate result
if cfg.run.save_intermediate:
    intermediate_file = get_output_file_name(step="s03", type="prep", description="zero_thresholded_imputed", ext="csv")
    df.to_csv(intermediate_file, index=False)
    print(f"Intermediate result, zero-thresholded and imputed, saved to {intermediate_file}")

Step 3: Imputed remaining '<LOQ' values with LOQ/2 for each peptide.
Intermediate result, zero-thresholded and imputed, saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s03_prep_zero_thresholded_imputed_20251209.csv


### Step 4: Shapiro-Wilk test for normality on each peptide across samples

In [115]:
def do_shapiro(df, groups):
    results = []
    for _, row in df.iterrows():
        for group, cols in groups.items():
            values = row[cols].astype(float).values
            if len(values) < 3:
                continue

            stat, pval = shapiro(values)
            _, (slope, intercept, r) = probplot(values)
            
            results.append({
                "peptide": row["Peptide"],
                "protein": row["Protein"],
                "group": group,

                "is_normal_shapiro": pval > 0.01,
                "is_normal_qq": r >= 0.990,

                "shapiro_stat": stat,
                "shapiro_pval": pval,

                "qq_slope": slope,
                "qq_intercept": intercept,
                "qq_rvalue": r
            })
    return results

# * Applied Shapiro-Wilk test on INITIAL peptide data (before log transformation)
# ? normality_df: peptide, protein, group, is_normal_shapiro, is_normal_qq, 
# ?               shapiro_stat, shapiro_pval, qq_slope, qq_intercept, qq_rvalue
normality_df = pd.DataFrame(do_shapiro(df, groups))
normality_df.sort_values(by=["peptide", "group"], inplace=True) # by=["group", "peptide"]

# * Count percentage of non-normal peptides
# ? normal: array of peptides and their groups that have is_normal_shapiro == True
normal = normality_df.loc[normality_df["is_normal_shapiro"], ["peptide", "group"]]
print(f"Number of normal group-peptides: %d out of %d (%.2f%%)" % (normal.shape[0], normality_df.shape[0], normal.shape[0] / normality_df.shape[0] * 100))

# * Peptides normal in all 3 groups (by Shapiro)
normal_all_3 = (
    normality_df.groupby("peptide")
    .agg(n_groups=("group", "nunique"), all_normal=("is_normal_shapiro", "all"))
    .query("n_groups == 3 and all_normal")
    .index.tolist()
)
print(f"Peptides normal in all 3 groups (Shapiro): %d" % len(normal_all_3))


# * Apply log10 transformation and re-test normality
df_log = df.copy()
df_log[value_cols] = np.log10(
    df[value_cols].astype(float).clip(lower=1e-12)
)
normality_df_log = pd.DataFrame(do_shapiro(df_log, groups))
normality_df_log.sort_values(by=["peptide", "group"], inplace=True)

# * Count percentage of non-normal peptides after log transformation
normal_log = normality_df_log.loc[normality_df_log["is_normal_shapiro"], ["peptide", "group"]]
print(f"Number of normal group-peptides (after log transformation): %d out of %d (%.2f%%)" % (normal_log.shape[0], normality_df_log.shape[0], normal_log.shape[0] / normality_df_log.shape[0] * 100))

# * Peptides normal in all 3 groups (by Shapiro)
normal_all_3 = (
    normality_df_log.groupby("peptide")
    .agg(n_groups=("group", "nunique"), all_normal=("is_normal_shapiro", "all"))
    .query("n_groups == 3 and all_normal")
    .index.tolist()
)
print(f"Peptides normal in all 3 groups (Shapiro): %d" % len(normal_all_3))

print(f"Step 4: Completed Shapiro-Wilk tests for each peptide within each group.")

if cfg.run.save_intermediate:
    normality_df.to_csv(
        get_output_file_name(step="s04", type="stats", description="shapiro_normality", ext="csv"),
        index=False
    )
    normality_df_log.to_csv(
        get_output_file_name(step="s04", type="stats", description="shapiro_normality_log_transformed", ext="csv"),
        index=False
    )
    print("Saved normality test results to CSV files.")

Number of normal group-peptides: 39 out of 120 (32.50%)
Peptides normal in all 3 groups (Shapiro): 0
Number of normal group-peptides (after log transformation): 75 out of 120 (62.50%)
Peptides normal in all 3 groups (Shapiro): 14
Step 4: Completed Shapiro-Wilk tests for each peptide within each group.
Saved normality test results to CSV files.


### Step 5: Doing statistical tests between groups

In [116]:
# TODO: condition for choosing test based on normality results
# * Using Kruskal-Wallis test for all peptides (non-parametric)
kw_results = []
group_names = list(groups.keys())
for _, row in df.iterrows():
    values_by_group = [row[groups[g]].astype(float).values for g in group_names]
    stat, pval = kruskal(*values_by_group)
    kw_results.append({
        "peptide": row["Peptide"],
        "protein": row["Protein"],
        "kw_stat": stat,
        "kw_pval": pval,
        "is_significant": pval < 0.01,
    })

kw_df = pd.DataFrame(kw_results)
kw_df.sort_values(by=["is_significant", "kw_pval", "peptide"], ascending=[False, True, True], inplace=True)


# * Using ANOVA for peptides normal in all 3 groups
anova_results = []
for peptide in normal_all_3:
    row = df.loc[df["Peptide"] == peptide].iloc[0]
    values_by_group = [row[groups[g]].astype(float).values for g in group_names]
    stat, pval = f_oneway(*values_by_group)
    anova_results.append({
        "peptide": row["Peptide"],
        "protein": row["Protein"],
        "anova_stat": stat,
        "anova_pval": pval,
        "is_significant": pval < 0.01,
    })
anova_df = pd.DataFrame(anova_results)
anova_df.sort_values(by=["is_significant", "anova_pval", "peptide"], ascending=[False, True, True], inplace=True)

print(f"Step 5: Statistical tests between groups done.")

if cfg.run.save_intermediate:
    kw_df.to_csv(
        get_output_file_name(step="s05", type="stats", description="kruskal_wallis_results", ext="csv"),
        index=False
    )
    print("Saved statistical test results to CSV files.")


Step 5: Statistical tests between groups done.
Saved statistical test results to CSV files.


### Step 6: Applying Bonferroni and Benjamini-Hochberg correction and detecting significant peptides


In [117]:
sig_kw = set(kw_df.loc[kw_df["is_significant"], "peptide"])
sig_anova = set(anova_df.loc[anova_df["is_significant"], "peptide"])

sig_peptides = sorted(sig_kw.union(sig_anova))
n_sig = len(sig_peptides)

kw_pvals = kw_df["kw_pval"].values

_, kw_pvals_bonf, _, _ = multipletests(kw_pvals, method="bonferroni")
_, kw_pvals_bh, _, _    = multipletests(kw_pvals, method="fdr_bh")

kw_df["kw_pval_bonf"] = kw_pvals_bonf
kw_df["kw_pval_fdr_bh"] = kw_pvals_bh

kw_df["is_significant_bonf"] = kw_df["kw_pval_bonf"] < 0.01
kw_df["is_significant_fdr"]  = kw_df["kw_pval_fdr_bh"] < 0.01
sig_kw_fdr = set(kw_df.loc[kw_df["is_significant_fdr"], "peptide"])
sig_peptides_fdr = sorted(sig_kw_fdr)

sig_kw_bonf = set(kw_df.loc[kw_df["is_significant_bonf"], "peptide"])
sig_peptides_bonf = sorted(sig_kw_bonf)
if n_sig == 0:
    print(f"Step 6: No significant peptides found. Skipping visualizations.")

print(f"Step 6: Corrections on the statistics done.")
print(f"Unique significant peptides from Kruskal/ANOVA after Bonferroni correction:", len(sig_peptides_bonf))
print(f"Unique significant peptides from Kruskal/ANOVA after FDR correction:", len(sig_peptides_fdr))

# if cfg.run.use_correction:
#     sig_peptides = sig_peptides_bonf

print(sig_peptides)
print(len(sig_peptides))

if cfg.run.save_intermediate:
    kw_df.to_csv(
        get_output_file_name(step="s06", type="stats", description="kruskal_wallis_results_with_corrections", ext="csv"),
        index=False
    )
    print("Saved Kruskal-Wallis results with corrections to CSV file.")

Step 6: Corrections on the statistics done.
Unique significant peptides from Kruskal/ANOVA after Bonferroni correction: 3
Unique significant peptides from Kruskal/ANOVA after FDR correction: 5
['ETLLQDFR', 'FRPDGLPK', 'GVCEETSGAYEK', 'LGNQEPGGQTALK', 'QVVAGLNFR', 'TVGSDTFYSFK', 'TYMLAFDVNDEK', 'WFYIASAFR', 'YVGGQEHFAHLLILR']
9
Saved Kruskal-Wallis results with corrections to CSV file.


### Step 7: Visualizations of the groups of peptides that are significant

**7.1:** Set colors for groups, got labels for peptides with their proteins


In [118]:
group_colors = {
    "PD":  "#636EFA",   # blue
    "RBD": "#EF553B",   # red
    "CON": "#00CC96",   # green
}

group_names = list(groups.keys())

peptide_to_protein = (
    df.loc[:, ["Peptide", "Protein"]]
    .dropna(subset=["Peptide"])
    .drop_duplicates(subset=["Peptide"], keep="first")
    .set_index("Peptide")["Protein"]
    .to_dict()
)

def _title_with_protein(peptide_name: str) -> str:
    protein = peptide_to_protein.get(peptide_name, "Unknown protein")
    return f"{protein} ({peptide_name})"

peptide_titles = [_title_with_protein(p) for p in sig_peptides]


**7.2**: Box plots

In [119]:
fig_box = make_subplots(
    rows=1,
    cols=len(sig_peptides),
    shared_yaxes=False,
    subplot_titles=peptide_titles
)
for col_idx, peptide in enumerate(sig_peptides, start=1):
    row_pep = df.loc[df["Peptide"] == peptide].iloc[0]
    for g in group_names:
        values = row_pep[groups[g]].astype(float).values
        fig_box.add_trace(
            go.Box(
                y=values,
                name=g,
                boxpoints="outliers",
                showlegend=(col_idx == 1),
                marker_color=group_colors.get(g),
            ),
            row=1,
            col=col_idx
        )
width_box = max(350 * len(sig_peptides), 800)
height_box = 400
fig_box.update_layout(
    title="Significant Peptides – Box Plots per Peptide (3 groups each)",
    width=width_box,
    height=height_box
)
# Axis labels for boxplots
fig_box.update_xaxes(title_text="Group")
fig_box.update_yaxes(title_text="Peptide intensity", row=1, col=1)
save_figure(
    fig_box,
    "s07",
    "significant_peptides_boxplots",
    cfg,
    OUTPUT_DIR
)
print(f"Step 7.2: Box plots done.")

Figure saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s07_fig_significant_peptides_boxplots_20251209.png
Step 7.2: Box plots done.


**7.3**: Histograms: median bars

In [120]:
fig_hist = make_subplots(
    rows=1,
    cols=len(sig_peptides),
    shared_yaxes=False,
    subplot_titles=peptide_titles
)
for col_idx, peptide in enumerate(sig_peptides, start=1):
    row_pep = df.loc[df["Peptide"] == peptide].iloc[0]
    for g in group_names:
        values = row_pep[groups[g]].astype(float).values
        values = values[~np.isnan(values)]
        if len(values) == 0:
            median_val = np.nan
        else:
            median_val = float(np.median(values))
        fig_hist.add_trace(
            go.Bar(
                x=[g],
                y=[median_val],
                name=g,
                marker_color=group_colors.get(g),
                showlegend=(col_idx == 1)
            ),
            row=1,
            col=col_idx
        )
fig_hist.update_layout(
    barmode="group"
)
width_hist = max(300 * len(sig_peptides), 300)
height_hist = 350
fig_hist.update_layout(
    title="Significant Peptides – Group Medians per Peptide (3 bars each)",
    width=width_hist,
    height=height_hist
)
# Axis labels for median-bar histograms
fig_hist.update_xaxes(title_text="Group")
fig_hist.update_yaxes(title_text="Median peptide intensity", row=1, col=1)
save_figure(
    fig_hist,
    "s07",
    "significant_peptides_median_barplots",
    cfg,
    OUTPUT_DIR
)
print(f"Step 7.3: Median bar plots done.")


Figure saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s07_fig_significant_peptides_median_barplots_20251209.png
Step 7.3: Median bar plots done.


**7.4**: Density distributions

In [121]:
fig_dist = make_subplots(
    rows=1,
    cols=len(sig_peptides),
    shared_yaxes=False,
    subplot_titles=peptide_titles
)
for col_idx, peptide in enumerate( sig_peptides, start=1):
    row_pep = df.loc[df["Peptide"] == peptide].iloc[0]

    all_vals = []
    group_vals = {}
    for g in group_names:
        vals = row_pep[groups[g]].astype(float).values
        vals = vals[~np.isnan(vals)]
        if len(vals) == 0:
            continue
        group_vals[g] = vals
        all_vals.append(vals)
    if not all_vals:
        continue
    all_vals = np.concatenate(all_vals)
    x_min, x_max = float(all_vals.min()), float(all_vals.max())
    if x_min == x_max:
        x_min -= 1e-6
        x_max += 1e-6
    xs = np.linspace(x_min, x_max, 200)

    for g in group_names:
        if g not in group_vals:
            continue
        vals = group_vals[g]
        if len(vals) < 2:
            continue
        kde = gaussian_kde(vals)
        ys = kde(xs)
        fig_dist.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                mode="lines",
                name=g,
                showlegend=(col_idx == 1),
                line=dict(color=group_colors.get(g)),
            ),
            row=1,
            col=col_idx
        )
width_dist = max(400 * len(sig_peptides), 400)
height_dist = 350
fig_dist.update_layout(
    title="Significant Peptides – Density Distributions per Peptide (3 groups each)",
    width=width_dist,
    height=height_dist
)
# Axis labels for density plots
fig_dist.update_xaxes(title_text="Peptide intensity")
fig_dist.update_yaxes(title_text="Density", row=1, col=1)
save_figure(
    fig_dist,
    "s07",
    "significant_peptides_density_distributions",
    cfg,
    OUTPUT_DIR
)
print(f"Step 7.4: Distribution plots saved.")


Figure saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s07_fig_significant_peptides_density_distributions_20251209.png
Step 7.4: Distribution plots saved.


### Step 8: Analysis of RBD subgroups of significant peptides

In [122]:
rbd_groups = {
    "A": [c for c in value_cols if c.endswith("RBDA")],
    "B": [c for c in value_cols if c.endswith("RBDB")],
    "C": [c for c in value_cols if c.endswith("RBDC")],
}

rbd_kw_results = []
rbd_descriptives = []

df_by_peptide = df.set_index("Peptide", drop=False)

for peptide in df_by_peptide.index.unique():
    row_pep = df_by_peptide.loc[peptide]
    if isinstance(row_pep, pd.DataFrame):
        row_pep = row_pep.iloc[0]

    subgroup_values = []
    for subgroup, cols in rbd_groups.items():
        vals = row_pep[cols].astype(float).values
        vals = vals[~np.isnan(vals)]
        subgroup_values.append(vals)
        rbd_descriptives.append({
            "peptide": peptide,
            "rbd_group": subgroup,
            "n": len(vals),
            "median": float(np.median(vals)),
            "mean": float(np.mean(vals)),
        })

    stat, pval = kruskal(*subgroup_values)
    rbd_kw_results.append({
        "peptide": peptide,
        "kw_stat": stat,
        "kw_pval": pval,
        "is_significant": pval < 0.01,
    })

    rbd_kw_df = pd.DataFrame(rbd_kw_results)
    rbd_kw_df.sort_values(by=["is_significant", "kw_pval", "peptide"], ascending=[False, True, True], inplace=True)

print("Step 8: RBD subgroup analysis completed. %d peptides tested." % (len(rbd_kw_results)))

print(rbd_kw_df)
if cfg.run.save_intermediate:
    rbd_kw_df.to_csv(
        get_output_file_name(step="s08", type="stats", description="rbd_subgroup_kruskal_wallis_results", ext="csv"),
        index=False
    )
    rbd_desc_df = pd.DataFrame(rbd_descriptives)
    rbd_desc_df.to_csv(
        get_output_file_name(step="s08", type="stats", description="rbd_subgroup_descriptives", ext="csv"),
        index=False
    )
    print("Saved RBD subgroup analysis results to CSV files.")

Step 8: RBD subgroup analysis completed. 40 peptides tested.
              peptide   kw_stat   kw_pval  is_significant
10       TYMLAFDVNDEK  5.141476  0.076479           False
9     NWGLSVYADKPETTK  4.316933  0.115502           False
23         NQVSLTCLVK  4.105532  0.128379           False
22            DTLMISR  3.758674  0.152691           False
11    YVGGQEHFAHLLILR  3.170786  0.204867           False
17      LGNQEPGGQTALK  3.125935  0.209513           False
25       GPSVFPLAPSSK  2.905769  0.233895           False
24  TTPPVLDSDGSFFLYSK  2.537839  0.281135           False
21          AKPALEDLR  2.504410  0.285874           False
30           GLPSSIEK  2.056991  0.357545           False
5          SVLGQLGITK  1.969363  0.373558           False
15          WFYIASAFR  1.956071  0.376049           False
29  TTPPVLDSDGSFFLYSR  1.944877  0.378160           False
20      DYVSQFEGSALGK  1.866560  0.393262           False
19            DFLQSLK  1.691204  0.429299           False
1          

In [123]:
print(rbd_kw_df[rbd_kw_df["is_significant"]])

Empty DataFrame
Columns: [peptide, kw_stat, kw_pval, is_significant]
Index: []


In [124]:
print(rbd_kw_df.loc[rbd_kw_df["peptide"].isin(sig_peptides)])

            peptide   kw_stat   kw_pval  is_significant
10     TYMLAFDVNDEK  5.141476  0.076479           False
11  YVGGQEHFAHLLILR  3.170786  0.204867           False
17    LGNQEPGGQTALK  3.125935  0.209513           False
15        WFYIASAFR  1.956071  0.376049           False
36     GVCEETSGAYEK  1.361120  0.506333           False
35         ETLLQDFR  1.048470  0.592008           False
7          FRPDGLPK  0.817822  0.664373           False
37        QVVAGLNFR  0.783527  0.675864           False
39      TVGSDTFYSFK  0.759433  0.684055           False


# PCA

In [125]:
try:
    data_matrix = df[value_cols].astype(float).T
    sample_ids = data_matrix.index.to_list()
    data_log = np.log10(data_matrix + 1.0e-6)
    if "Peptide" in df.columns:
        raw_labels = df["Peptide"].astype(str).fillna("Unknown").tolist()
    else:
        raw_labels = [f"feature_{i}" for i in range(data_matrix.shape[1])]
    if len(raw_labels) != data_matrix.shape[1]:
        raw_labels = [f"feature_{i}" for i in range(data_matrix.shape[1])]
    data_matrix.columns = raw_labels

    # Pareto scaling
    col_means = data_log.mean(axis=0)
    col_std = data_log.std(axis=0, ddof=1).replace(0, np.nan)
    pareto_scale = np.sqrt(col_std).replace(0, np.nan)
    data_pareto = (data_log - col_means) / pareto_scale
    data_pareto = data_pareto.dropna(axis=1, how="any")
    retained_features = data_pareto.columns.to_list()

    # Number of components is set here
    pca = PCA(n_components=0.95) # automatically calculate all components
    pca_scores = pca.fit_transform(data_pareto.values)

    sample_groups = []
    for sid in sample_ids:
        if sid.startswith("BIOPD"): sample_groups.append("PD")
        elif sid.startswith("RBD"): sample_groups.append("RBD")
        elif sid.startswith("CON"): sample_groups.append("CON")
        else: sample_groups.append("Other")

    pca_df = pd.DataFrame({
        "sample": sample_ids,
        "group": sample_groups,
        **{f"PC{i+1}": pca_scores[:, i] for i in range(pca.n_components_)}
    })
    print(pca_df.head())

    component_labels = [f"PC{i+1}" for i in range(pca.n_components_)]
    loadings_df = pd.DataFrame(
        pca.components_.T,
        columns=component_labels,
        index=retained_features,
    )

    # loadings_export = loadings_df.reset_index().rename(columns={"index": "feature"})
    # loadings_export.to_csv(os.path.join(OUTPUT_DIR, "pca_loadings.csv"), index=False)
    loadings_df["loading_magnitude"] = np.sqrt(loadings_df["PC1"] ** 2 + loadings_df["PC2"] ** 2)
    top_n_loadings = min(20, len(loadings_df))
    top_loadings = loadings_df.nlargest(top_n_loadings, "loading_magnitude") if top_n_loadings else loadings_df

    print("PCA performed successfully.")

    if cfg.run.save_intermediate:
        pca_df.to_csv(
            get_output_file_name(step="s09", type="viz", description="pca_scores", ext="csv"),
            index=False
        )
        loadings_df.to_csv(
            get_output_file_name(step="s09", type="viz", description="pca_loadings", ext="csv"),
            index=True
        )
        print("Saved PCA scores and loadings to CSV files.")
except Exception as e:
    print(f"Error during PCA computation: {e}")


       sample group       PC1       PC2       PC3       PC4       PC5  \
0  BIOPD01-PD    PD -7.235638  1.128659  0.487541  1.515903 -0.105528   
1  BIOPD02-PD    PD  0.441854  0.889148  0.041314  0.089671  0.125016   
2  BIOPD03-PD    PD  0.178178  0.203484  0.401832 -0.454481 -0.112173   
3  BIOPD04-PD    PD  0.388716  1.134879  0.449513 -0.940297 -0.172990   
4  BIOPD05-PD    PD  0.431733 -0.599999  0.439188 -0.180371  0.330722   

        PC6       PC7       PC8       PC9      PC10      PC11      PC12  \
0 -2.358134 -9.195681  1.241966  0.173682 -0.596435 -0.286541  0.722732   
1 -0.739859  0.163158 -0.928677 -0.751154 -0.627971  0.058459  0.258179   
2  0.237961 -0.296588 -0.720740 -0.536039  0.165714  0.383040 -0.325623   
3 -0.167911  0.413705  0.607859 -0.152130  0.215700 -0.039528 -0.603797   
4  1.114072  0.163863  0.550158 -0.179840 -1.189720 -0.233458 -0.633632   

       PC13      PC14      PC15      PC16      PC17  
0 -0.703420 -0.028908  0.351896 -0.163869  0.338307  
1 

### PCA scores plot

In [126]:
try:
    group_colors_pca = {
        "PD":  "#636EFA",
        "RBD": "#EF553B",
        "CON": "#00CC96",
        "Other": "#AB63FA",
    }

    fig_pca = go.Figure()
    for grp in sorted(pca_df["group"].unique()):
        grp_df = pca_df[pca_df["group"] == grp]
        fig_pca.add_trace(
            go.Scatter(
                x=grp_df["PC1"],
                y=grp_df["PC2"],
                mode="markers",
                name=grp,
                marker=dict(
                    size=10,
                    opacity=0.8,
                    color=group_colors_pca.get(grp, "#888888"),
                ),
                text=grp_df["sample"],
                hovertemplate="Sample: %{text}<br>PC1: %{x:.3f}<br>PC2: %{y:.3f}<extra></extra>",
            )
        )

    fig_pca.update_layout(
        title="PCA Score scatter",
        xaxis_title=f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)",
        yaxis_title=f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)",
        width=800,
        height=700,
    )
    save_figure(
        fig_pca,
        "s09",
        "pca_score_scatter",
        cfg,
        OUTPUT_DIR
    )
    print(f"Step 9: PCA plotting done.")
except Exception as e:
    print(f"Error during PCA computation or plotting: {e}")

Figure saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s09_fig_pca_score_scatter_20251209.png
Step 9: PCA plotting done.


## PCA loadings plot

In [127]:
try:
    if not loadings_df.empty:
        fig_loadings = go.Figure()
        for feature, row in top_loadings.iterrows():
            fig_loadings.add_shape(
                type="line",
                x0=0,
                y0=0,
                x1=row["PC1"],
                y1=row["PC2"],
                line=dict(color="#BBBBBB", width=1),
            )
        fig_loadings.add_trace(
            go.Scatter(
                x=top_loadings["PC1"],
                y=top_loadings["PC2"],
                mode="markers+text",
                text=top_loadings.index,
                textposition="top center",
                marker=dict(size=9, color="#333333"),
                hovertemplate="Feature: %{text}<br>PC1 loading: %{x:.3f}<br>PC2 loading: %{y:.3f}<extra></extra>",
                showlegend=False,
            )
        )
        fig_loadings.update_layout(
            title="PCA Loadings (PC1 vs PC2)",
            xaxis_title="PC1 loading",
            yaxis_title="PC2 loading",
            xaxis=dict(zeroline=True, zerolinewidth=1, zerolinecolor="#333"),
            yaxis=dict(zeroline=True, zerolinewidth=1, zerolinecolor="#333"),
            width=800,
            height=700,
        )
        save_figure(
            fig_loadings,
            "s09",
            "pca_loadings_pc1_pc2",
            cfg,
            OUTPUT_DIR
        )
        print(f"Step 9: PCA loadings plotting done.")
except Exception as e:
    print(f"Error during PCA loadings plotting: {e}")

Figure saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s09_fig_pca_loadings_pc1_pc2_20251209.png
Step 9: PCA loadings plotting done.


## PCA biplot

In [128]:
try:
    score_max = float(np.max(np.abs(pca_scores[:, :2]))) if pca_scores.size else 1.0
    loading_max = float(np.max(np.abs(top_loadings[["PC1", "PC2"]].values))) if not top_loadings.empty else 1.0
    loading_max = loading_max if loading_max != 0 else 1.0
    biplot_scale = (score_max / loading_max) * 0.7 if loading_max else 1.0
    scaled_loadings = top_loadings[["PC1", "PC2"]] * biplot_scale

    fig_biplot = go.Figure()
    for grp in sorted(pca_df["group"].unique()):
        grp_df = pca_df[pca_df["group"] == grp]
        fig_biplot.add_trace(
            go.Scatter(
                x=grp_df["PC1"],
                y=grp_df["PC2"],
                mode="markers",
                name=f"{grp} samples",
                marker=dict(
                    size=10,
                    opacity=0.8,
                    color=group_colors_pca.get(grp, "#888888"),
                ),
                text=grp_df["sample"],
                hovertemplate="Sample: %{text}<br>PC1: %{x:.3f}<br>PC2: %{y:.3f}<extra></extra>",
            )
        )

    for feature, row in scaled_loadings.iterrows():
        fig_biplot.add_shape(
            type="line",
            x0=0,
            y0=0,
            x1=row["PC1"],
            y1=row["PC2"],
            line=dict(color="#444444", width=1),
        )

    fig_biplot.add_trace(
        go.Scatter(
            x=scaled_loadings["PC1"],
            y=scaled_loadings["PC2"],
            mode="markers+text",
            text=scaled_loadings.index,
            textposition="top center",
            marker=dict(color="#444444", size=9, symbol="diamond"),
            name="Feature loadings",
            hovertemplate="Feature: %{text}<br>Scaled PC1 loading: %{x:.3f}<br>Scaled PC2 loading: %{y:.3f}<extra></extra>",
        )
    )

    fig_biplot.update_layout(
        title="PCA Biplot (PC1 vs PC2)",
        xaxis_title=f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)",
        yaxis_title=f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)",
        width=900,
        height=750,
    )
    save_figure(
        fig_biplot,
        "s09",
        "pca_biplot_pc1_pc2",
        cfg,
        OUTPUT_DIR
    )
except Exception as e:
    print(f"Error during PCA biplot plotting: {e}")

print("Step 9: PCA plots and scores saved successfully.")

Figure saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s09_fig_pca_biplot_pc1_pc2_20251209.png
Step 9: PCA plots and scores saved successfully.


In [129]:
try:
    use_only_significant = False

    if use_only_significant:
        peptide_subset = sig_peptides
    else:
        peptide_subset = df["Peptide"].dropna().unique().tolist()

    pca_df_raw = df[df["Peptide"].isin(peptide_subset)]
    data_matrix = pca_df_raw.set_index("Peptide")[value_cols].astype(float)

    data_log = np.log10(data_matrix + 1e-6)
    data_log = data_log.replace([np.inf, -np.inf], np.nan)

    data_log = data_log.dropna(axis=0, how="all")

    row_means = data_log.mean(axis=1)
    data_log = data_log.T.fillna(row_means).T

    # Pareto scaling
    row_std = data_log.std(axis=1, ddof=1)
    pareto_scale = np.sqrt(row_std)
    pareto_scale[pareto_scale == 0] = 1.0
    data_centered = data_log.sub(row_means, axis=0)
    data_pareto = data_centered.div(pareto_scale, axis=0).dropna(axis=0, how="any")

    # PCA (rows = peptides, cols = samples)
    # n_components = min(data_pareto.shape[0], data_pareto.shape[1])
    pca = PCA(n_components=2)
    pca_scores = pca.fit_transform(data_pareto.values)

    peptides_final = data_pareto.index.tolist()
    proteins_final = (
        df.loc[df["Peptide"].isin(peptides_final), ["Peptide", "Protein"]]
        .drop_duplicates("Peptide")
        .set_index("Peptide")["Protein"]
        .to_dict()
    )

    peptide_pca_df = pd.DataFrame({
        "peptide": peptides_final,
        "protein": [proteins_final.get(p, "Unknown") for p in peptides_final],
        "PC1": pca_scores[:, 0],
        "PC2": pca_scores[:, 1],
    })

    # Coloring by protein
    protein_colors = {}
    unique_proteins = peptide_pca_df["protein"].unique()
    for i, pr in enumerate(unique_proteins):
        protein_colors[pr] = f"hsl({(i * 47) % 360},70%,50%)"

    fig_pep_pca = go.Figure()
    for pr in unique_proteins:
        temp = peptide_pca_df[peptide_pca_df["protein"] == pr]
        fig_pep_pca.add_trace(
            go.Scatter(
                x=temp["PC1"],
                y=temp["PC2"],
                mode="markers",
                name=pr,
                marker=dict(size=8, opacity=0.8, color=protein_colors[pr]),
                text=temp["peptide"],
                hovertemplate="Peptide: %{text}<br>PC1: %{x:.3f}<br>PC2: %{y:.3f}<extra></extra>",
            )
        )

    fig_pep_pca.update_layout(
        title="Peptide-level PCA (log10 + Pareto scaled)",
        xaxis_title=f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)",
        yaxis_title=f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)",
        width=900,
        height=800,
    )
    save_figure(
        fig_pep_pca,
        "s09",
        "peptide_level_pca",
        cfg,
        OUTPUT_DIR
    )
except Exception as e:
    print(f"Error during peptide-level PCA: {e}")

Figure saved to ./results/run_000_20251209-165259/p02_peptide_analysis_000_default_s09_fig_peptide_level_pca_20251209.png
